## Walkthrough of Data Science - Traveler

### * Goal: Predict the country that users will make their first booking in, based on some basic user profile data.


#### [1] Pre-processing: Assessing and analyzing data, cleaning, transforming and adding new features
#### [2] Learning model: Constructing and testing learning model
#### [3] Post-processing: Creating final predictions


# LAB 1 CODE

In [15]:
##Exploring Traveler data
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline 

print("Reading data...")
train_file = "/home/ubuntu/datamininglab/DMALab-master/Lab1/train_users_2.csv"
df_train = pd.read_csv(train_file, header = 0,index_col=None)

test_file = "/home/ubuntu/datamininglab/DMALab-master/Lab1/test_users.csv"
df_test = pd.read_csv(test_file, header = 0,index_col=None)

# Combining into one dataset for cleaning
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
print("Reading data...completed")

# Fixing date formats in Pandas - to_datetime
## Change dates to specific format
print("Fixing timestamps...")
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format='%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format='%Y%m%d%H%M%S')
print("Fixing timestamps...completed")

## Removing date_first_booking column
df_all.drop('date_first_booking', axis = 1, inplace = True)
print("Droped date_first_booking column...")

import numpy as np

## Remove outliers function - [1]
def remove_outliers(df, column, min_val, max_val):
    col_values = df[column].values
    df[column] = np.where(np.logical_or(col_values<=min_val, col_values>=max_val), np.NaN, col_values)
    return df

## Fixing age column - [2]
print("Fixing age column...")
df_all = remove_outliers(df = df_all, column = 'age', min_val = 15, max_val = 90)
df_all['age'].fillna(-1, inplace = True)
print("Fixing age column...completed")

# Other column missing value - Fill first_affiliate_tracked column
print("Filling first_affiliate_tracked column...")
df_all['first_affiliate_tracked'].fillna(-1, inplace=True)
print("Filling first_affiliate_tracked column...completed")

df_all.head()

Populating the interactive namespace from numpy and matplotlib
Reading data...
Reading data...completed
Fixing timestamps...
Fixing timestamps...completed
Droped date_first_booking column...
Fixing age column...
Fixing age column...completed
Filling first_affiliate_tracked column...
Filling first_affiliate_tracked column...completed


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in less_equal
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in greater_equal


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,-1.0,NDF,2010-06-28,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2009-03-19 04:32:55
1,seo,google,38.0,NDF,2011-05-25,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2009-05-23 17:48:09
2,direct,direct,56.0,US,2010-09-28,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2009-06-09 23:12:47
3,direct,direct,42.0,other,2011-12-05,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2009-10-31 06:01:29
4,direct,direct,41.0,US,2010-09-14,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,2009-12-08 06:11:05


# data transformation and feature extraction

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

train_data = pd.read_csv("/home/ubuntu/datamininglab/DMALab-master/Lab1/train_users_2.csv")
test_data = pd.read_csv("/home/ubuntu/datamininglab/DMALab-master/Lab1/test_users.csv")
df_all = pd.concat((train_data, test_data), ignore_index = True)

def convert_to_binary(df, col_to_convert):
    categories = list(df[col_to_convert].drop_duplicates())
    print(categories)
    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace("/", "_").replace("-", "").lower()
        col_name = col_to_convert[:5] + '_' + cat_name[:10]
        df[col_name] = 0
        df.loc[(df[col_to_convert] == category), col_name] = 1
    return df

print("One Hot Encoding of Categorical Data")
cols_to_convert = ["gender", 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']

for col in cols_to_convert:
    df_all = convert_to_binary(df = df_all, col_to_convert = col)
    df_all.drop(col, axis = 1, inplace = True)
   
print("One Hot Encoding Categorical Data completed")    

One Hot Encoding of Categorical Data
['-unknown-', 'MALE', 'FEMALE', 'OTHER']
['facebook', 'basic', 'google', 'weibo']
[0, 3, 2, 1, 24, 8, 6, 5, 10, 25, 12, 4, 16, 15, 20, 21, 23, 14]
['en', 'fr', 'de', 'es', 'it', 'pt', 'zh', 'ko', 'ja', 'ru', 'pl', 'el', 'sv', 'nl', 'hu', 'da', 'id', 'fi', 'no', 'tr', 'th', 'cs', 'hr', 'ca', 'is', '-unknown-']
['direct', 'seo', 'other', 'sem-non-brand', 'content', 'sem-brand', 'remarketing', 'api']
['direct', 'google', 'other', 'craigslist', 'facebook', 'vast', 'bing', 'meetup', 'facebook-open-graph', 'email-marketing', 'yahoo', 'padmapper', 'gsp', 'wayn', 'naver', 'baidu', 'yandex', 'daum']
['untracked', 'omg', nan, 'linked', 'tracked-other', 'product', 'marketing', 'local ops']
['Web', 'Moweb', 'iOS', 'Android']
['Mac Desktop', 'Windows Desktop', 'iPhone', 'Other/Unknown', 'Desktop (Other)', 'Android Tablet', 'iPad', 'Android Phone', 'SmartPhone (Other)']
['Chrome', 'IE', 'Firefox', 'Safari', '-unknown-', 'Mobile Safari', 'Chrome Mobile', 'RockMelt

In [22]:
df_all.head()

,age,country_destination,date_account_created,date_first_booking,id,timestamp_first_active,gende_unknown,gende_male,gende_female,gende_other,...,first_theworld_b,first_slimbrowse,first_epic,first_stainless,first_googlebot,first_outlook_20,first_icedragon,first_ibrowse,first_nintendo_b,first_uc_browser
0,NaN,NDF,2010-06-28,NaN,gxn3p5htnn,20090319043255,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38.0,NDF,2011-05-25,NaN,820tgsjxq7,20090523174809,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,56.0,US,2010-09-28,2010-08-02,4ft3gnwmtx,20090609231247,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,42.0,other,2011-12-05,2012-09-08,bjjt8pjhuk,20091031060129,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,41.0,US,2010-09-14,2010-02-18,87mebub9p4,20091208061105,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_all.shape[1]

158

In [27]:
df_all.count()
df_all["timestamp_first_active"] = pd.to_datetime(df_all["timestamp_first_active"], format = "%Y%m%d%H%M%S")

df_all["date_account_created"] = pd.to_datetime(df_all["date_account_created"], format = "%Y-%m-%d")

In [30]:
print ("adding new fields")
df_all["day_account_created"]=df_all["date_account_created"].dt.weekday
df_all["month_account_created"]=df_all["date_account_created"].dt.month
df_all["quarter_account_created"]=df_all["date_account_created"].dt.quarter
df_all["year_account_created"]=df_all["date_account_created"].dt.year
df_all["hour_first_active"]=df_all["timestamp_first_active"].dt.hour
df_all["day_first_active"]=df_all["timestamp_first_active"].dt.weekday
df_all["month_first_active"]=df_all["timestamp_first_active"].dt.month
df_all["quarter_first_active"]=df_all["timestamp_first_active"].dt.quarter
df_all["year_first_active"]=df_all["timestamp_first_active"].dt.year
df_all["created_less_Active"]=(df_all["date_account_created"]-df_all["timestamp_first_active"]).dt.days
print("adding new flds completed")

print("droping fields..")
columns_to_drop=["date_account_created","timestamp_first_active","date_first_booking","country_destination"]
for column in columns_to_drop:
    if column in df_all.columns:
        df_all.drop(column,axis=1,inplace=True)
print("drping fld completed")



adding new fields
adding new flds completed
droping fields..
drping fld completed


In [31]:
df_all.count()

age                        158681
id                         275547
gende_unknown              275547
gende_male                 275547
gende_female               275547
gende_other                275547
signu_facebook             275547
signu_basic                275547
signu_google               275547
signu_weibo                275547
signu_0                    275547
signu_3                    275547
signu_2                    275547
signu_1                    275547
signu_24                   275547
signu_8                    275547
signu_6                    275547
signu_5                    275547
signu_10                   275547
signu_25                   275547
signu_12                   275547
signu_4                    275547
signu_16                   275547
signu_15                   275547
signu_20                   275547
signu_21                   275547
signu_23                   275547
signu_14                   275547
langu_en                   275547
langu_fr      

#HW:with dummy variables created by one-hot-encoding are we introducing multicollinearity>
#if yes...can we use PCA to reduce the dimensionality down to prevent the model from being misled>??
#hw: how worried abt multicollinearity you need to be if you are 1) building a regression model and ii) concerned with prediction
#accuracy??



#why multicollinearity not checked in modern stackexchange

# understanding session data

In [36]:

import numpy as np
df_sessions=pd.read_csv("/home/ubuntu/datamininglab/sessions.csv")

In [39]:
df_sessions.head(10)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0
5,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,7703.0
6,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,115.0
7,d1mm9tcy42,personalize,data,wishlist_content_update,Windows Desktop,831.0
8,d1mm9tcy42,index,view,view_search_results,Windows Desktop,20842.0
9,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,683.0


In [38]:
print("determining primary devices")
sessions_device =df_sessions.loc[:,["user_id","device_type","secs_elapsed"]]
aggregated_lvl1=sessions_device.groupby(["user_id","device_type"],as_index=False,sort=False).aggregate(np.sum)
idx=aggregated_lvl1.groupby(["user_id"],sort=False)["secs_elapsed"].transform(max)==aggregated_lvl1["secs_elapsed"]
df_sessions_primary=pd.DataFrame(aggregated_lvl1.loc[idx,["user_id","device_type","secs_elapsed"]])

df_sessions_primary.rename(columns={"device_type":"primary_device","secs_elapsed":"primary_secs"},inplace=True)

df_sessions_primary = convert_to_binary(df = df_sessions_primary, col_to_convert='primary_device')
df_sessions_primary.drop('primary_device', axis = 1, inplace = True)
print("Determining primary device completed")


determining primary devices
['Windows Desktop', 'Mac Desktop', 'iPhone', 'iPad Tablet', '-unknown-', 'Android App Unknown Phone/Tablet', 'Linux Desktop', 'Tablet', 'Chromebook', 'Android Phone', 'iPodtouch', 'Blackberry', 'Windows Phone', 'Opera Phone']
Determining primary device completed


In [40]:
##loopimg through action columns